# Day 3 – SQL via Python: NYC School Data Exploration

### 1. Imports

In [1]:
# importing libraries
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
import psycopg2

## 2. Connecting to Database

In [2]:
# DB connection setup using hardcoded credentials (for onboarding only)
conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="a9Am7Yy5r9_T7h4OF2GN",
    host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
    port="5432",
    sslmode="require"
)
cur = conn.cursor()

In [3]:
# Load high_school_directory data and save to dataframe

query = "SELECT * FROM nyc_schools.high_school_directory;"
df_high_schools_directory = pd.read_sql(query, conn)
df_high_schools_directory.head()

/var/folders/gq/3d_53n2d05n2_n592s2dd_mr0000gn/T/ipykernel_34989/2117804111.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_high_schools_directory = pd.read_sql(query, conn)


,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,number_programs,Location 1,Community Board,Council District,Census Tract,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,NaN,NaN,...,1,"{'latitude': '40.601989336', 'longitude': '-73...",14,31,100802,20529,51,3,47,59
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,NaN,NaN,...,1,"{'latitude': '40.593593811', 'longitude': '-73...",13,47,306,17616,21,2,45,35
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,NaN,NaN,...,1,"{'latitude': '40.692133704', 'longitude': '-73...",3,36,291,18181,69,2,49,52
3,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,NaN,NaN,...,1,"{'latitude': '40.822303765', 'longitude': '-73...",9,18,16,11611,58,5,31,26
4,03M485,Fiorello H. LaGuardia High School of Music & A...,Manhattan,M485,212-496-0700,212-724-5748,9.0,12,NaN,NaN,...,6,"{'latitude': '40.773670507', 'longitude': '-73...",7,6,151,12420,20,4,19,12


In [4]:
print("Shape of DataFrame:", df_high_schools_directory.shape)
print("Columns in DataFrame:", df_high_schools_directory.columns.tolist())

Shape of DataFrame: (435, 105)
Columns in DataFrame: ['dbn', 'school_name', 'borough', 'building_code', 'phone_number', 'fax_number', 'grade_span_min', 'grade_span_max', 'expgrade_span_min', 'expgrade_span_max', 'start_time', 'end_time', 'priority01', 'priority02', 'priority03', 'priority04', 'priority05', 'priority06', 'priority07', 'priority08', 'priority09', 'priority10', 'location', 'phone_number2', 'school_email', 'website', 'subway', 'bus', 'grades2018', 'finalgrades', 'total_students', 'extracurricular_activities', 'school_sports', 'attendance_rate', 'pct_stu_enough_variety', 'pct_stu_safe', 'school_accessibility_description', 'directions1', 'requirement1', 'requirement2', 'requirement3', 'requirement4', 'requirement5', 'program1', 'code1', 'interest1', 'method1', 'seats9ge1', 'grade9gefilledflag1', 'grade9geapplicants1', 'seats9swd1', 'grade9swdfilledflag1', 'grade9swdapplicants1', 'campus_name', 'building_borough', 'building_location', 'latitude', 'longitude', 'community_board

In [5]:
# load school_demographics data and save to dataframe

query = "SELECT * FROM nyc_schools.school_demographics;"
df_school_demographics = pd.read_sql(query, conn)
df_school_demographics.head()

/var/folders/gq/3d_53n2d05n2_n592s2dd_mr0000gn/T/ipykernel_34989/2264191815.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_school_demographics = pd.read_sql(query, conn)


,dbn,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,...,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
0,01M015,P.S. 015 ROBERTO CLEMENTE,20052006,89.4,NaN,281,15,36,40,33,...,74,26.3,189,67.3,5,1.8,158,56.2,123,43.8
1,01M015,P.S. 015 ROBERTO CLEMENTE,20062007,89.4,NaN,243,15,29,39,38,...,68,28.0,153,63.0,4,1.6,140,57.6,103,42.4
2,01M015,P.S. 015 ROBERTO CLEMENTE,20072008,89.4,NaN,261,18,43,39,36,...,77,29.5,157,60.2,7,2.7,143,54.8,118,45.2
3,01M015,P.S. 015 ROBERTO CLEMENTE,20082009,89.4,NaN,252,17,37,44,32,...,75,29.8,149,59.1,7,2.8,149,59.1,103,40.9
4,01M015,P.S. 015 ROBERTO CLEMENTE,20092010,,96.5,208,16,40,28,32,...,67,32.2,118,56.7,6,2.9,124,59.6,84,40.4


In [6]:
print("Shape of DataFrame:", df_school_demographics.shape)
print("Columns in DataFrame:", df_school_demographics.columns.tolist())


Shape of DataFrame: (243, 38)
Columns in DataFrame: ['dbn', 'Name', 'schoolyear', 'fl_percent', 'frl_percent', 'total_enrollment', 'prek', 'k', 'grade1', 'grade2', 'grade3', 'grade4', 'grade5', 'grade6', 'grade7', 'grade8', 'grade9', 'grade10', 'grade11', 'grade12', 'ell_num', 'ell_percent', 'sped_num', 'sped_percent', 'ctt_num', 'selfcontained_num', 'asian_num', 'asian_per', 'black_num', 'black_per', 'hispanic_num', 'hispanic_per', 'white_num', 'white_per', 'male_num', 'male_per', 'female_num', 'female_per']


## 3. School Distribution by Borough

Question: How many schools are there in each borough?

#### Approach
Simple aggregation query:
1. **COUNT(DISTINCT dbn)** to count total schools per borough
2. **GROUP BY** borough to aggregate at borough level
3. **ORDER BY** num_schools (descending) to show highest counts first

#### Results

| Borough        | Number of Schools |
|----------------|-------------------|
| Brooklyn       | 121               |
| Bronx          | 118               |
| Manhattan      | 106               |
| Queens         | 80                |
| Staten Island  | 10                |

**Total Schools:** 435 high schools across NYC

#### Interpretation

**Key Findings:**
- **Brooklyn leads** with 121 schools (27.8% of all NYC high schools), followed closely by the Bronx (118 schools, 27.1%)
- **Manhattan** has 106 schools despite being densely populated, reflecting space constraints
- **Queens** is underrepresented with only 80 schools (18.4%), despite being NYC's largest borough by area
- **Staten Island** has significantly fewer schools (10, only 2.3%), consistent with its lower population density and more suburban character

**Distribution Pattern:**
The school distribution roughly correlates with population density and urbanization, with the four major boroughs (Brooklyn, Bronx, Manhattan, Queens) containing 97.7% of all high schools, while Staten Island's suburban nature results in far fewer but potentially larger schools.

In [7]:
# School Distribution

query = """
SELECT 
    borough,
    COUNT(DISTINCT dbn) as num_schools
FROM nyc_schools.high_school_directory
GROUP BY borough
ORDER BY num_schools DESC
"""

school_distribution = pd.read_sql(query, conn)
print("School Distribution by Borough:")
print(school_distribution)

School Distribution by Borough:
         borough  num_schools
0       Brooklyn          121
1          Bronx          118
2      Manhattan          106
3         Queens           80
4  Staten Island           10


/var/folders/gq/3d_53n2d05n2_n592s2dd_mr0000gn/T/ipykernel_34989/524876053.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  school_distribution = pd.read_sql(query, conn)


## 4. Average Percentage of English Language Learners (ELL) per Borough
Question: What is the average % of English Language Learners (ELL) per borough?

#### Approach
Used SQL query with:
1. **LEFT JOIN** between `high_school_directory` and `school_demographics` on `dbn` (school ID)
2. **AVG()** aggregation function to calculate mean `ell_percent` per borough
3. **GROUP BY** borough to get borough-level averages
4. **ORDER BY** avg_ell_percent (ascending) for readability

#### Results

| Borough        | Avg ELL % |
|----------------|-----------|
| Manhattan      | 7.57%     |
| Brooklyn       | NaN       |
| Queens         | NaN       |
| Bronx          | NaN       |
| Staten Island  | NaN       |

#### Interpretation

**Critical Issue - Data Mismatch:**
- Only **Manhattan** has a calculated average (7.57% ELL students)
- All other boroughs return **NaN** (Not a Number), indicating **no matching data** in the join

**Possible Causes:**
1. **DBN mismatch**: The `dbn` values in `school_demographics` don't match those in `high_school_directory` for most boroughs
2. **Missing data**: `school_demographics` may only contain Manhattan schools or has incomplete coverage
3. **Null values**: `ell_percent` column might be NULL for non-Manhattan schools




In [8]:
query = """
SELECT 
    hsd.borough,
    AVG(sd.ell_percent) as avg_ell_percent
FROM nyc_schools.high_school_directory hsd
LEFT JOIN nyc_schools.school_demographics sd
ON hsd.dbn = sd.dbn
GROUP BY hsd.borough
ORDER BY avg_ell_percent ASC

""" 
df_ell_distribution = pd.read_sql(query, conn)
df_ell_distribution

/var/folders/gq/3d_53n2d05n2_n592s2dd_mr0000gn/T/ipykernel_34989/3393431448.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ell_distribution = pd.read_sql(query, conn)


,borough,avg_ell_percent
0,Manhattan,7.5725
1,Brooklyn,NaN
2,Queens,NaN
3,Staten Island,NaN
4,Bronx,NaN


In [9]:
#join dr_high_schools_directory and df_school_demographics on dbn to analyze ell_percent by borough using python
merge_df = df_high_schools_directory.merge(df_school_demographics, on="dbn", how="left")
merge_df.head()

,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,03M485,Fiorello H. LaGuardia High School of Music & A...,Manhattan,M485,212-496-0700,212-724-5748,9.0,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
print(merge_df.shape)
print(df_high_schools_directory.shape
      , df_school_demographics.shape    )

(468, 142)
(435, 105) (243, 38)


In [11]:
# check value counts of ell_percent column
merge_df["borough"].value_counts()


borough
Manhattan        139
Brooklyn         121
Bronx            118
Queens            80
Staten Island     10
Name: count, dtype: int64

In [12]:
merge_df[merge_df['ell_percent'].notnull()]

,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
19,01M450,East Side Community School,Manhattan,M060,212-460-8467,212-260-9657,6.0,12,NaN,NaN,...,138.0,27.5,299.0,59.7,28.0,5.6,244.0,48.7,257.0,51.3
20,01M450,East Side Community School,Manhattan,M060,212-460-8467,212-260-9657,6.0,12,NaN,NaN,...,161.0,28.6,337.0,60.0,30.0,5.3,296.0,52.7,266.0,47.3
21,01M450,East Side Community School,Manhattan,M060,212-460-8467,212-260-9657,6.0,12,NaN,NaN,...,149.0,26.9,335.0,60.5,29.0,5.2,288.0,52.0,266.0,48.0
22,01M450,East Side Community School,Manhattan,M060,212-460-8467,212-260-9657,6.0,12,NaN,NaN,...,145.0,26.6,326.0,59.7,33.0,6.0,285.0,52.2,261.0,47.8
23,01M450,East Side Community School,Manhattan,M060,212-460-8467,212-260-9657,6.0,12,NaN,NaN,...,151.0,27.0,321.0,57.4,31.0,5.5,299.0,53.5,260.0,46.5
24,01M450,East Side Community School,Manhattan,M060,212-460-8467,212-260-9657,6.0,12,NaN,NaN,...,152.0,25.4,339.0,56.6,58.0,9.7,319.0,53.3,280.0,46.7
25,01M450,East Side Community School,Manhattan,M060,212-460-8467,212-260-9657,6.0,12,NaN,NaN,...,143.0,23.9,331.0,55.4,62.0,10.4,327.0,54.7,271.0,45.3
122,01M292,Henry Street School for International Studies,Manhattan,M056,212-406-9411,212-406-9417,6.0,12,NaN,NaN,...,106.0,36.1,133.0,45.2,10.0,3.4,160.0,54.4,134.0,45.6
123,01M292,Henry Street School for International Studies,Manhattan,M056,212-406-9411,212-406-9417,6.0,12,NaN,NaN,...,137.0,31.6,208.0,47.9,14.0,3.2,241.0,55.5,193.0,44.5
124,01M292,Henry Street School for International Studies,Manhattan,M056,212-406-9411,212-406-9417,6.0,12,NaN,NaN,...,158.0,30.7,272.0,52.8,12.0,2.3,281.0,54.6,234.0,45.4


In [13]:
# What is the average % of English Language Learners (ELL) per borough?
avg_ell_by_borough = merge_df.groupby("borough")["ell_percent"].mean().reset_index()
avg_ell_by_borough.sort_values(by="ell_percent", ascending=False)

,borough,ell_percent
2,Manhattan,7.5725
0,Bronx,NaN
1,Brooklyn,NaN
3,Queens,NaN
4,Staten Island,NaN


## 5. Schools per Borough by Special Education Percentage

Question: Using the data from the school demographics and high school directory, write a query to find the top 3 schools in each borough with the highest percentage of special education students (sped_percent)


#### Approach
Here a SQL query with window functions is used to:
1. **JOIN** `high_school_directory` and `school_demographics` tables on `dbn` (school ID)
2. **RANK** schools within each borough using `ROW_NUMBER()` partitioned by borough and ordered by `sped_percent` (descending)
3. **FILTER** to select only the top 3 ranked schools per borough
4. Excluded schools with null `sped_percent` values

#### Results

**Unexpected Finding:**
The query returned only **one school** (East Side Community School, Manhattan) with **three different years** of data, rather than three different schools per borough.

| DBN    | School Name                | Borough   | SPED % | SPED # | Total Enrollment | Rank |
|--------|----------------------------|-----------|--------|--------|------------------|------|
| 01M450 | East Side Community School | Manhattan | 28.8%  | 161    | 559              | 1    |
| 01M450 | East Side Community School | Manhattan | 27.7%  | 166    | 599              | 2    |
| 01M450 | East Side Community School | Manhattan | 26.7%  | 146    | 546              | 3    |

### Interpretation

**Issue Identified:**
- The `school_demographics` table contains **multiple years of data** per school (evident from `schoolyear` column)
- The query is ranking **school-year combinations** instead of unique schools
- This explains why the same school appears 3 times with varying SPED percentages



In [14]:
# SQL query to find top 3 schools per borough by sped_percent
query = """
WITH ranked_schools AS (
    SELECT 
        hsd.dbn,
        hsd.school_name,
        hsd.borough,
        sd.sped_percent,
        sd.sped_num,
        sd.total_enrollment,
        ROW_NUMBER() OVER (PARTITION BY hsd.borough ORDER BY sd.sped_percent DESC) as rank
    FROM nyc_schools.high_school_directory hsd
    INNER JOIN nyc_schools.school_demographics sd ON hsd.dbn = sd.dbn
    WHERE sd.sped_percent IS NOT NULL
)
SELECT 
    dbn,
    school_name,
    borough,
    sped_percent,
    sped_num,
    total_enrollment,
    rank
FROM ranked_schools
WHERE rank <= 3
ORDER BY borough, rank
"""

# Execute query
top_sped_schools = pd.read_sql(query, conn)

# Display results
print("Top 3 Schools per Borough by Special Education Percentage:")
print(top_sped_schools)


/var/folders/gq/3d_53n2d05n2_n592s2dd_mr0000gn/T/ipykernel_34989/926454978.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  top_sped_schools = pd.read_sql(query, conn)


Top 3 Schools per Borough by Special Education Percentage:
      dbn                 school_name    borough  sped_percent  sped_num  \
0  01M450  East Side Community School  Manhattan          28.8       161   
1  01M450  East Side Community School  Manhattan          27.7       166   
2  01M450  East Side Community School  Manhattan          26.7       146   

   total_enrollment  rank  
0               559     1  
1               599     2  
2               546     3  


In [15]:
#using merge_df find the top 3 schools in each borough with the highest percentage of special education students (sped_percent)
merge_df[merge_df['sped_percent'].notnull()].groupby('borough').apply(lambda x: x.nlargest(3, 'sped_percent'))[['school_name','dbn','sped_percent','borough']]

/var/folders/gq/3d_53n2d05n2_n592s2dd_mr0000gn/T/ipykernel_34989/2475412570.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merge_df[merge_df['sped_percent'].notnull()].groupby('borough').apply(lambda x: x.nlargest(3, 'sped_percent'))[['school_name','dbn','sped_percent','borough']]


school_name     dbn  sped_percent    borough
borough                                                                  
Manhattan 23  East Side Community School  01M450          28.8  Manhattan
          24  East Side Community School  01M450          27.7  Manhattan
          22  East Side Community School  01M450          26.7  Manhattan

## 6. Key Insights from SQL Analysis

### 1. Data Quality Issues Identified
- Only Manhattan schools successfully matched between `high_school_directory` and `school_demographics` tables, resulting in NaN values for all other boroughs in ELL analysis
- `school_demographics` contains multiple years per school, causing ranking queries to return duplicate schools across different years instead of unique schools

### 2. School Distribution Patterns
- Brooklyn (121) and Bronx (118) dominate with 55% of all NYC high schools, while Staten Island has only 10 schools (2.3%)


### 3. Special Education & ELL Data
- Special education analysis only returned Manhattan data with East Side Community School showing 26.7-28.8% SPED enrollment across years
- 7.57% English Language Learner percentage - cannot generalize to other boroughs due to data gaps

